# Kurdish Letter OCR - Model Testing Notebook

## Overview
This notebook loads the trained Kurdish letter OCR model and tests it on new images.

## Contents
1. **Setup** - Load model and dependencies
2. **Prediction Function** - Single image classification
3. **Batch Testing** - Test multiple images at once
4. **Visualization** - Display results with confidence scores

In [2]:
import os
import cv2
import numpy as np
import torch
import torch.nn as nn
from sklearn.preprocessing import LabelEncoder
import glob

print("Libraries loaded successfully!")

Libraries loaded successfully!


## Configuration & Model Definition

Set up parameters and define the CNN model architecture.

In [3]:
# Configuration parameters (must match training parameters)
IMG_SIZE = 64
CHANNELS = 1
MODEL_PATH = "kurdish_letter_model_pytorch.pth"

# Device setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Categories (must match training)
CATEGORIES = [
    {"folder": "EEE_letters", "prefix": "eee_letter_"},
    {"folder": "LLL_letters", "prefix": "lll_letter_"},
    {"folder": "OOO_letters", "prefix": "ooo_letter_"},
    {"folder": "RRR_letters", "prefix": "rrr_letter_"},
    {"folder": "VVV_letters", "prefix": "vvv_letter_"}
]

EXTENSIONS = [".png", ".jpg", ".jpeg", ".bmp"]

class KurdishCNN(nn.Module):
    """
    Convolutional Neural Network for Kurdish letter classification.
    
    Architecture:
    - 3 convolutional layers with ReLU activation
    - MaxPooling after each convolution
    - 2 fully connected layers with dropout
    - Output: 5 classes (Kurdish letters)
    """
    def __init__(self, num_classes):
        super(KurdishCNN, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1)
        self.relu = nn.ReLU()

        # Calculate flattened features
        with torch.no_grad():
            dummy_input = torch.zeros(1, 1, IMG_SIZE, IMG_SIZE)
            x = self.pool(self.relu(self.conv1(dummy_input)))
            x = self.pool(self.relu(self.conv2(x)))
            x = self.pool(self.relu(self.conv3(x)))
            num_flat_features = x.numel() // x.shape[0]

        # Fully connected layers
        self.fc1 = nn.Linear(num_flat_features, 64)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(64, num_classes)

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.pool(self.relu(self.conv3(x)))
        x = x.view(x.size(0), -1)
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x


Using device: cpu


## Model Loading

Load the trained model weights and prepare for inference.

In [4]:
# Initialize label encoder with training categories
le = LabelEncoder()
le.fit(np.array([cat["folder"] for cat in CATEGORIES]))

print(f"Classes available: {le.classes_}\n")

# Load the model
model = KurdishCNN(num_classes=len(CATEGORIES)).to(device)

# Load saved weights
if os.path.exists(MODEL_PATH):
    model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
    print(f"✓ Model loaded from: {MODEL_PATH}")
else:
    print(f"✗ Model file not found: {MODEL_PATH}")
    print("Please train the model first using Kurdish_Letter_OCR_CNN.ipynb")

# Set model to evaluation mode (disables dropout, batch norm, etc.)
model.eval()
print("✓ Model set to evaluation mode\n")


Classes available: ['EEE_letters' 'LLL_letters' 'OOO_letters' 'RRR_letters' 'VVV_letters']

✓ Model loaded from: kurdish_letter_model_pytorch.pth
✓ Model set to evaluation mode



## Prediction Functions

Functions to predict class for single or batch images.

In [5]:
def predict_single_image(image_path):
    """
    Predict the class of a single image.
    
    Parameters:
    -----------
    image_path : str
        Path to the image file
    
    Returns:
    --------
    tuple
        (predicted_class, confidence, probabilities)
        - predicted_class: str - The predicted Kurdish letter class
        - confidence: float - Confidence score (0-100)
        - probabilities: np.array - Confidence for each class
    """
    try:
        # Read image in grayscale
        img_array = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        
        if img_array is None:
            print(f"✗ Error: Could not load image from {image_path}")
            return None, None, None
        
        # Resize to model input size
        img_resized = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
        
        # Normalize (0-1 range)
        img_normalized = img_resized / 255.0
        
        # Convert to tensor (batch_size=1, channels=1)
        img_tensor = torch.tensor(img_normalized, dtype=torch.float32).unsqueeze(0).unsqueeze(0)
        
        # Move to device
        img_tensor = img_tensor.to(device)
        
        # Forward pass (no gradient computation)
        with torch.no_grad():
            output = model(img_tensor)
            probabilities = torch.softmax(output, dim=1)[0].cpu().numpy()
            predicted_idx = torch.argmax(output, dim=1).item()
            confidence = probabilities[predicted_idx] * 100
        
        # Get class name
        predicted_class = le.classes_[predicted_idx]
        
        return predicted_class, confidence, probabilities
    
    except Exception as e:
        print(f"✗ Error processing image: {str(e)}")
        return None, None, None


def display_prediction(image_path, predicted_class, confidence, probabilities):
    """
    Display prediction results in a formatted table.
    
    Parameters:
    -----------
    image_path : str
        Path to the image
    predicted_class : str
        Predicted class name
    confidence : float
        Confidence score
    probabilities : np.array
        Probabilities for all classes
    """
    print(f"\n{'='*70}")
    print(f"File: {os.path.basename(image_path)}")
    print(f"{'='*70}")
    print(f"Predicted Class: {predicted_class}")
    print(f"Confidence: {confidence:.2f}%\n")
    
    print("Class Probabilities:")
    print(f"{'Class':<20} {'Probability':<15} {'Confidence %':<15}")
    print("-" * 50)
    
    for idx, class_name in enumerate(le.classes_):
        prob = probabilities[idx]
        percent = prob * 100
        bar_length = int(percent / 5)
        bar = "█" * bar_length + "░" * (20 - bar_length)
        print(f"{class_name:<20} {prob:<15.4f} {bar} {percent:6.2f}%")
    
    print(f"{'='*70}\n")


def test_multiple_images(folder_path=None, file_list=None):
    """
    Test multiple images at once.
    
    Parameters:
    -----------
    folder_path : str, optional
        Test all images in a folder
    file_list : list, optional
        Test specific image files
    """
    if folder_path:
        # Find all images in folder
        image_files = []
        for ext in EXTENSIONS:
            image_files.extend(glob.glob(os.path.join(folder_path, f"*{ext}")))
            image_files.extend(glob.glob(os.path.join(folder_path, f"*{ext.upper()}")))
        image_files.sort()
    elif file_list:
        image_files = file_list
    else:
        print("Please provide either folder_path or file_list")
        return
    
    if not image_files:
        print("No images found")
        return
    
    print(f"\n{'='*70}")
    print(f"TESTING {len(image_files)} IMAGE(S)")
    print(f"{'='*70}\n")
    
    for img_path in image_files:
        predicted_class, confidence, probabilities = predict_single_image(img_path)
        
        if predicted_class is not None:
            display_prediction(img_path, predicted_class, confidence, probabilities)


## Usage Examples

Use the functions to test images.

In [6]:
# Example 1: Test a single image
# Uncomment and modify the path to test a specific image
# image_path = "path/to/your/image.jpg"
# predicted_class, confidence, probabilities = predict_single_image(image_path)
# if predicted_class:
#     display_prediction(image_path, predicted_class, confidence, probabilities)


# Example 2: Test all JPG files in current directory
print("Testing JPG files in current directory...\n")
jpg_files = glob.glob("*.jpg") + glob.glob("*.JPG") + glob.glob("*.jpeg") + glob.glob("*.JPEG")

if jpg_files:
    test_multiple_images(file_list=sorted(jpg_files))
else:
    print("No JPG files found in current directory")


# Example 3: Test images in a specific folder
# Uncomment to test images in a folder
# test_multiple_images(folder_path="EEE_letters")


# Example 4: Test specific image files
# Uncomment to test specific files
# test_multiple_images(file_list=["image1.jpg", "image2.png", "image3.jpg"])


Testing JPG files in current directory...


TESTING 2 IMAGE(S)


File: testing_image_1.JPG
Predicted Class: OOO_letters
Confidence: 99.98%

Class Probabilities:
Class                Probability     Confidence %   
--------------------------------------------------
EEE_letters          0.0002          ░░░░░░░░░░░░░░░░░░░░   0.02%
LLL_letters          0.0000          ░░░░░░░░░░░░░░░░░░░░   0.00%
OOO_letters          0.9998          ███████████████████░  99.98%
RRR_letters          0.0000          ░░░░░░░░░░░░░░░░░░░░   0.00%
VVV_letters          0.0000          ░░░░░░░░░░░░░░░░░░░░   0.00%


File: testing_image_2.JPG
Predicted Class: RRR_letters
Confidence: 99.91%

Class Probabilities:
Class                Probability     Confidence %   
--------------------------------------------------
EEE_letters          0.0000          ░░░░░░░░░░░░░░░░░░░░   0.00%
LLL_letters          0.0001          ░░░░░░░░░░░░░░░░░░░░   0.01%
OOO_letters          0.0003          ░░░░░░░░░░░░░░░░░░░░   0.03%
RRR_l